In [1]:
print("hi")

hi


In [2]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers


c:\Users\Prajwal R\anaconda3\envs\mchatbot\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
PINECONE_API_KEY = "pcsk_76BbC2_twsaGT1Bc2pP5hjBuQHocCny5Wct7z4uB6mg6D6xbdami75Q2wi9vnR6wXvpe7"
PINECONE_API_ENV = "gcp-starter"

In [4]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [5]:
extracted_data = load_pdf("data/")

In [6]:
 #extracted_data

In [7]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [8]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 5859


In [9]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

embeddings = download_hugging_face_embeddings()

C:\Users\Prajwal R\AppData\Local\Temp\ipykernel_15344\2261448754.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [10]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [11]:
# text_chunks

In [12]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [13]:
embeddings = download_hugging_face_embeddings()

In [14]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [15]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [16]:
# query_result

In [17]:
import os
from pinecone import Pinecone, ServerlessSpec

# Read your keys from environment variables or set directly
api_key = os.getenv("PINECONE_API_KEY", "your_key_here")
env = os.getenv("PINECONE_API_ENV", "your_env_here")

# Create Pinecone client instance
pc = Pinecone(api_key=api_key)

# Your index name
index_name = "medical-chatbot"

# Create index if not exists
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,  # Set your embedding dimension here
        metric="cosine",
        spec=ServerlessSpec(
            cloud="gcp",
            region="us-central1"
        )
    )

# Then proceed with LangChain Pinecone vectorstore as you did


In [18]:
from langchain_community.vectorstores import Pinecone as LangchainPinecone

In [19]:
#If we already have an index we can load it like this
# ...existing code...
docsearch = LangchainPinecone.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)
# ...existing code...
query = "What are Allergies"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

Result [Document(metadata={'page': 135.0, 'source': 'data\\Medical_book.pdf'}, page_content='Purpose\nAllergy is a reaction of the immune system. Nor-\nmally, the immune system responds to foreign microor-\nganisms and particles, like pollen or dust, by producing\nspecific proteins called antibodies that are capable of\nbinding to identifying molecules, or antigens, on the\nforeign organisms. This reaction between antibody and\nantigen sets off a series of reactions designed to protect\nthe body from infection. Sometimes, this same series of'), Document(metadata={'page': 129.0, 'source': 'data\\Medical_book.pdf'}, page_content='reaction. Allergic rhinitis is characterized by an itchy,\nrunny nose, often with a scratchy or irritated throat due\nto post-nasal drip. Inflammation of the thin membrane\ncovering the eye (allergic conjunctivitis) causes redness,\nirritation, and increased tearing in the eyes. Asthma caus-\nes wheezing, coughing, and shortness of breath. Symp-\ntoms of food al

In [20]:
# ...existing code...
docsearch = LangchainPinecone.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)
# ...existing code...

In [21]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [22]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [23]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [24]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [25]:
from langchain import PromptTemplate

prompt_template = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs = {"prompt": PROMPT}

In [ ]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])

C:\Users\Prajwal R\AppData\Local\Temp\ipykernel_15344\4216317267.py:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  result=qa({"query": user_input})


Response :  Acne is a common skin disease characterized by pimples on the face, chest, and back that occur when the pores of the skin become clogged with oil, dead skin cells, and bacteria.
